In [7]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Read data

In [9]:
dat = pd.read_csv("df_from_R.csv.gz")

<ipython-input-9-f06570a1f288>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv("df_from_R.csv.gz")


In [10]:
dat = dat.rename(columns = {"donor": "Individual", "sample": "Sample", 
                            "pos":"Pos", "ref":"Ref", "alt":"Alt"})
dat["Individual"] = dat.Individual.str[0:7]
dat["Chr"] = "chr" + dat["chrom"].astype(str)
dat["Tissue_CellType"] = "Liver"
dat["Compartment"] = "WGS"
dat["Study"] = "Ng2021"

In [11]:
dat = tools.keep_snps(dat).reset_index(drop="True")
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]

In [12]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [13]:
dat["chrom_id"] = dat["Chr"] + dat["Pos"].astype(str) + dat["Alt"]

In [14]:
sdat = dat[dat.Individual=="PD36714"]

In [15]:
trees = sdat.groupby(["id", "Sample"]).size().reset_index()
trees.columns = ["id", "Sample", "n_muts"]

In [11]:
pmeta2 = pd.read_excel("41586_2021_3974_MOESM3_ESMM/Supplementary Table 2.xlsx", skiprows=2)
coverages = pmeta2.set_index("Sample")["Coverage"].to_dict()
burdens = dat.groupby("Sample")["id"].size().to_dict()
seen = []
miss = []
for k,v in burdens.items():
    if k in coverages:
        seen.append(k)
    else:
        miss.append(k)

In [14]:
dat.groupby("chrom_id").size().sort_values()

chrom_id
chr10100001407T      1
chr369436449A        1
chr369429085G        1
chr36942771T         1
chr369425064T        1
                  ... 
chr7145784901G      58
chr944189966C       70
chr1428368928C      87
chr562231849A       95
chr3192734974T     105
Length: 1946591, dtype: int64

In [17]:
len(dat.groupby("Sample").size())

1585

In [62]:
dat[col_order].to_csv("Ng2021.csv.gz", compression="gzip", index=False)

Metafile 

In [2]:
pmeta = pd.read_excel("41586_2021_3974_MOESM3_ESMM/Supplementary Table 1.xlsx", skiprows=4)
pmeta.assign(Study="Ng2021", Compartment="WGS", Tissue_CellType="Liver")
pmeta = pmeta.rename(columns={"Patient ID": "Individual", "Age at \noperation (yrs)":"Age"})

In [3]:
pmeta.replace("NORMAL","Normal")
pmeta["Status"] = pmeta.Aetiology

In [16]:
stat = pmeta.set_index("Individual")["Status"].to_dict()
cell2indv = dat.set_index("Sample")["Individual"].to_dict()
ages = pmeta.set_index("Individual")["Age"].to_dict()

In [27]:
smoking = pmeta.set_index("Individual")["Smoking \nstatus"].replace("Current", "smoker").replace("Never", "non-smoker").replace("Ex", "ex-smoker").to_dict()

In [28]:
l = list(dat.Sample.unique())
meta = pd.DataFrame(l)
meta.columns = ["Sample"]
meta["Study"] = "Ng2021"
meta["Tissue_CellType"] = "Liver"
meta["Individual"] = meta.Sample.map(cell2indv)
meta["Age"] = meta["Individual"].map(ages)
meta["Status"] = meta["Individual"].map(stat)
meta["Compartment"] = "WGS"
meta["Smoking"] = meta["Individual"].map(smoking)

In [29]:
meta = meta.replace("NORMAL", "Normal")

In [31]:
meta[col_meta].to_csv("Ng2021.tab.gz",sep="\t", index=False)